# Install / Import / Config

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
from dotenv import load_dotenv
load_dotenv()
BASE_DIR = os.getenv("BASE_DIR")

In [3]:
import edurel.utils.llm as llmu
import edurel.utils.duckdb as ddbu
import edurel.utils.mermaid as mmu

from edurel.widgets.mermaid_viz import MermaidVisualizer

DB_DIR = f"{BASE_DIR}/databases/"

# Database

In [4]:
con = ddbu.mem_con(DB_DIR + "db-company_en")
schema_plain = ddbu.schema(con)
schema_yaml = ddbu.schema_yaml(con)
schema_mermaid = mmu.schema_to_mermaid(schema_yaml)

In [6]:
# print(ddbu.schema(con))
# con.close()

In [6]:
viz = MermaidVisualizer(schema_mermaid, width="100%", height="500px")
viz.display() 

# LLM

In [ ]:
deepseek32_llm_stats = llmu.chat_llm_stats(llmu.DEEPSEEK32)
gemini3pro_llm_stats = llmu.chat_llm_stats(llmu.GEMINI3PRO)
glm46_llm_stats = llmu.chat_llm_stats(llmu.GLM46)
gpt41mini_llm_stats = llmu.chat_llm_stats(llmu.GPT41MINI)
opus41_llm_stats = llmu.chat_llm_stats(llmu.OPUS41)


In [7]:
ollama = llmu.chat_ollama(llmu.ARCTICTEXT2SQL)

## Q1

### EN

In [8]:
question = """
create a SQL query that outputs the following columns:
- eid, lastname, hiredate from table employee
- entry year per employee 
sorted by eid
"""

### ARK

In [9]:
sql = llmu.chat_text_to_sql(ollama, schema_plain, question)
print(sql)

The user wants to extract specific information about employees including their ID, last name, and hire date. Additionally, they want a computed column for the year of employment based on the hire date. The sorting should be done in ascending order based on employee IDs.
</think>
<action>
Now, let's translate this into :
1. Select columns `eid`, `lastname`, and `hiredate` from the table `Employee`.
2. Add a computed column for the year of employment using the function `strftime('%Y', hiredate)`. This will extract the year part from the date.
3. Order the results by `eid`.

The  query is as follows:

SELECT 
  EID AS eid, 
  LastName AS lastname, 
  HireDate AS hiredate,
  strftime('%Y', HireDate) AS entry_year
FROM Employee
ORDER BY EID;


This solution directly addresses the user's request without unnecessary complexity or extraneous joins. It focuses on the `Employee` table and extracts only the required columns.
</action>


In [12]:
ddbu.sql_print(con, sql)

┌───────┬──────────┬────────────┬────────────┐
│  eid  │ lastname │  hiredate  │ entry_year │
│ int32 │ varchar  │    date    │  varchar   │
├───────┼──────────┼────────────┼────────────┤
│   101 │ Patil    │ 2000-05-01 │ 2000       │
│   102 │ Durmaz   │ 2005-07-01 │ 2005       │
│   103 │ Blaschke │ 2002-11-01 │ 2002       │
│   104 │ Stone    │ 2006-06-01 │ 2006       │
│   105 │ Dalal    │ 2018-02-02 │ 2018       │
│   106 │ Li       │ 2002-12-01 │ 2002       │
│   107 │ Nguyen   │ 2006-07-01 │ 2006       │
│   108 │ Sanchez  │ 2014-04-01 │ 2014       │
│   109 │ Umarani  │ 2006-07-01 │ 2006       │
│   110 │ Ortega   │ 2005-09-02 │ 2005       │
│   111 │ Doshi    │ 2010-01-02 │ 2010       │
│   112 │ Singh    │ 2012-03-01 │ 2012       │
│   113 │ Jadhav   │ 2001-08-01 │ 2001       │
│   114 │ Popov    │ 2009-03-02 │ 2009       │
│   115 │ Kumar    │ 2013-05-01 │ 2013       │
│   116 │ Krause   │ 2011-08-01 │ 2011       │
│   117 │ Oezdem   │ 2014-08-01 │ 2014       │
│   118 │ Oke

### GEMINI3PRO3

In [20]:
sql = llmu.chat_text_to_sql(gemini3pro_llm_stats, schema_plain, question)
print(sql)


SELECT
  t1.pid,
  t1.name,
  t3.matrnr
FROM person AS t1
JOIN personenrolle AS t2
  ON t1.pid = t2.person_pid
JOIN studentin AS t3
  ON t2.prid = t3.prid
ORDER BY
  t1.pid;



In [21]:
ddbu.sql_print(con, sql)

┌───────┬────────────────┬─────────┐
│  pid  │      name      │ matrnr  │
│ int32 │    varchar     │ varchar │
├───────┼────────────────┼─────────┤
│     5 │ Max Mustermann │ 1001    │
│     6 │ Lisa Wagner    │ 1002    │
│     7 │ Tom Fischer    │ 1003    │
│     8 │ Sarah Meyer    │ 1004    │
│     9 │ Jan Becker     │ 1005    │
│    10 │ Nina Schulz    │ 1006    │
└───────┴────────────────┴─────────┘



### GLM46

In [ ]:
sql = llmu.chat_text_to_sql(glm46_llm_stats, schema_plain, question)
print(sql)


SELECT p.pid, p.name, s.matrnr
FROM person p
JOIN personenrolle pr ON p.pid = pr.person_pid
JOIN studentin s ON pr.prid = s.prid
ORDER BY p.pid;



In [14]:
ddbu.sql_print(con, sql)

┌───────┬────────────────┬─────────┐
│  pid  │      name      │ matrnr  │
│ int32 │    varchar     │ varchar │
├───────┼────────────────┼─────────┤
│     5 │ Max Mustermann │ 1001    │
│     6 │ Lisa Wagner    │ 1002    │
│     7 │ Tom Fischer    │ 1003    │
│     8 │ Sarah Meyer    │ 1004    │
│     9 │ Jan Becker     │ 1005    │
│    10 │ Nina Schulz    │ 1006    │
└───────┴────────────────┴─────────┘



### GPT41MINI

In [ ]:
sql = llmu.chat_text_to_sql(gpt41mini_llm_stats, schema_plain, question)
print(sql)


SELECT p.pid, p.name, s.matrnr
FROM person p
JOIN studentin s ON p.pid = s.prid
ORDER BY p.pid;



In [22]:
ddbu.sql_print(con, sql)

┌───────┬────────────────┬─────────┐
│  pid  │      name      │ matrnr  │
│ int32 │    varchar     │ varchar │
├───────┼────────────────┼─────────┤
│     5 │ Max Mustermann │ 1001    │
│     6 │ Lisa Wagner    │ 1002    │
│     7 │ Tom Fischer    │ 1003    │
│     8 │ Sarah Meyer    │ 1004    │
│     9 │ Jan Becker     │ 1005    │
│    10 │ Nina Schulz    │ 1006    │
└───────┴────────────────┴─────────┘



### OPUS41

In [ ]:
sql = llmu.chat_text_to_sql(opus41_llm_stats, schema_plain, question)
print(sql)


SELECT p.pid, p.name, s.matrnr
FROM person p
JOIN personenrolle pr ON p.pid = pr.person_pid
JOIN studentin s ON pr.prid = s.prid
ORDER BY p.pid;



In [11]:
ddbu.sql_print(con, sql)

┌───────┬────────────────┬─────────┐
│  pid  │      name      │ matrnr  │
│ int32 │    varchar     │ varchar │
├───────┼────────────────┼─────────┤
│     5 │ Max Mustermann │ 1001    │
│     6 │ Lisa Wagner    │ 1002    │
│     7 │ Tom Fischer    │ 1003    │
│     8 │ Sarah Meyer    │ 1004    │
│     9 │ Jan Becker     │ 1005    │
│    10 │ Nina Schulz    │ 1006    │
└───────┴────────────────┴─────────┘



## Q2

In [7]:
question = """
erstelle eine SQL-abfrage, die folgende spalten ausgibt:
- name aus tabelle person
- matrnr aus tabelle studentin
- raumnr aus tabelle raum
für die studentin mit matrnr 1001
und die räume, in den sie kurse belegt hat
sortiert nach raumnr
"""

### GLM46

In [25]:
sql = llmu.chat_text_to_sql(glm46_llm_stats, schema_plain, question)
print(sql)


SELECT p.name, s.matrnr, r.raumnr
FROM studentin s
JOIN belegung b ON s.prid = b.studentin_prid
JOIN kursbt kb ON b.kursbt_kbtid = kb.kbtid
JOIN raum r ON kb.raum_rid = r.rid
JOIN personenrolle pr ON s.prid = pr.prid
JOIN person p ON pr.person_pid = p.pid
WHERE s.matrnr = '1001'
ORDER BY r.raumnr;



In [ ]:
ddbu.sql_print(con, sql)

┌────────────────┬─────────┬─────────┐
│      name      │ matrnr  │ raumnr  │
│    varchar     │ varchar │ varchar │
├────────────────┼─────────┼─────────┤
│ Max Mustermann │ 1001    │ A101    │
│ Max Mustermann │ 1001    │ A102    │
│ Max Mustermann │ 1001    │ A102    │
│ Max Mustermann │ 1001    │ B201    │
│ Max Mustermann │ 1001    │ B201    │
│ Max Mustermann │ 1001    │ C301    │
│ Max Mustermann │ 1001    │ C301    │
│ Max Mustermann │ 1001    │ D401    │
│ Max Mustermann │ 1001    │ D401    │
│ Max Mustermann │ 1001    │ E501    │
├────────────────┴─────────┴─────────┤
│ 10 rows                  3 columns │
└────────────────────────────────────┘



## Q3

In [10]:
question = """
erstelle eine SQL-abfrage, die folgende spalten ausgibt:
- name aus tabelle person
- matrnr aus tabelle studentin
für alle studenten die im sommersemester 2023 kurse belegt haben,
die am montag stattfinden
"""

### GLM46

In [11]:
sql = llmu.chat_text_to_sql(glm46_llm_stats, schema_plain, question)
print(sql)


SELECT p.name, s.matrnr
FROM person p
JOIN personenrolle pr ON p.pid = pr.person_pid
JOIN studentin s ON s.prid = pr.prid
JOIN belegung b ON s.prid = b.studentin_prid
JOIN kursbt kbt ON b.kursbt_kbtid = kbt.kbtid
JOIN kurs k ON kbt.kurs_kid = k.kid
JOIN semester sem ON k.semester_sid = sem.sid
JOIN termin t ON kbt.termin_tid = t.tid
JOIN wochentermin wt ON t.tid = wt.tid
JOIN tag tag ON wt.tag_tid = tag.tid
WHERE sem.beginn >= '2023-04-01' AND sem.ende <= '2023-09-30'
AND tag.bez = 'Montag';



In [12]:
ddbu.sql_print(con, sql)

┌─────────┬─────────┐
│  name   │ matrnr  │
│ varchar │ varchar │
├─────────┴─────────┤
│      0 rows       │
└───────────────────┘

